## ETL Development

This file serves as as draft for designing & development of project ETLs - transformations & preprocessing of data for modeling. The ETL should be part of runnable  
pipeline for model encapsulation & enviroment isolation purposes (leak of statistics).

Includes:
- Transformation ETLs
- Encoding ETLs

ETLs must be part of pipeline to ensure statistical calculation only on train dataset.

Objetive of the ETL pipeline is to transform input dataset into model ready form.

In [72]:
import os
import sys

sys.dont_write_bytecode =True

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings('ignore')

loading data

In [73]:
DATA_DIR = './Data/'
FILE = 'spotify_tracks_kaggle_weekly.csv'

In [74]:
data = pd.read_csv(DATA_DIR + FILE)

columns to be dropped:

- track_id : ID
- artwork_url : url data - not intended for modelling purposes
- track_url : url data - not intended for modelling purposes
- track_name : user-friendly ID - not intended for modelling purposes (could be used in NLP analysis, however that will not be subject of this project)

In [75]:
DROP_COLUMNS = ['track_id', 'artwork_url', 'track_url', 'track_name']

In [76]:
data = data.drop(DROP_COLUMNS, axis=1, errors='ignore')

In [77]:
data.head()

,artist_name,year,popularity,album_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,language
0,Anirudh Ravichander,2024,59,"Leo Das Entry (From ""Leo"")",0.0241,0.753,97297.0,0.970,0.055300,8.0,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,Tamil
1,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,AAO KILLELLE,0.0851,0.780,207369.0,0.793,0.000000,10.0,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,Tamil
2,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,Mayakiriye Sirikiriye (Orchestral EDM),0.0311,0.457,82551.0,0.491,0.000000,2.0,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,Tamil
3,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,Scene Ah Scene Ah (Experimental EDM Mix),0.2270,0.718,115831.0,0.630,0.000727,7.0,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,Tamil
4,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,Gundellonaa X I Am a Disco Dancer (Mashup),0.0153,0.689,129621.0,0.748,0.000001,7.0,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,Tamil


### Encoding

Due to the nature of the dataset multiple categorical (numerical as well) columns will require specific encoding method to convey information correctly.

- Categorical data:
    - artist_name
    - album_name
    - language


- Numerical data:
    - key
    - mode

- Columns which can be encoded by OneHot principle:
    - language
    - artist_name (maybe but avg. artist popularity could be as well)

Defining ETL model structure

In [78]:
from sklearn.base import BaseEstimator, TransformerMixin

In [79]:
class ETL(BaseEstimator, TransformerMixin):
    '''Custom ETL model, abstract class, when creating an ETL use class inheritance and overwrite
       self.transform method with custom transformation, must return transformed X.'''

    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X

Artist & Albums COLUMN encoding

- problem: multiple artists in one string -> separated by ','
- solutions:
    - nested average: sum(averages of artists) / len(artists)
    - frequency

8491 unique aritst names ... OneHot/Dummy encoding is not feasible

In [80]:
data['artist_name'].str.split(',').explode().nunique()

8491

In [81]:
data['artist_name_split'] = data['artist_name'].str.split(',')

schema

In [82]:
data.explode('artist_name_split').groupby('artist_name_split')['popularity'].mean()

artist_name_split
 "Lebo M." Morake           2.666667
 'Thindukallu' Poovitha     0.000000
 (Rap) Devan               13.000000
 (Rap) Rajeev              40.500000
 *NSYNC                    16.088889
                             ...    
كورال سفينة الحب            1.000000
へちもでぃすこ                     0.000000
まどんなZ                       0.000000
チームまどんなちゃん                  0.000000
微微微笑                        0.000000
Name: popularity, Length: 8491, dtype: float64

Dedicated COLUMN ETL

In [83]:
class ArtistEncoding(ETL):

    SEPARATOR = ','
    TARGET_COLUMN = 'artist_name'
    Y_COLUMN = 'popularity'

    def transform(self, X, y=None):
        X_new = X.copy()

        # popularity aggregation method
        X_new = self.aggregate_popularity(X_new)

        return X_new
    

    def aggregate_popularity(self, X):
        
        X['artist_name_split'] = X[self.TARGET_COLUMN].str.split(self.SEPARATOR)
        
        artist_popularity = X.explode('artist_name_split').groupby('artist_name_split')[self.Y_COLUMN].mean()

        def encode(artists, values):
            popularities = [values.get(artist, 0) for artist in artists]
            return sum(popularities) / len(popularities) if popularities else 0


        X['artist_name_encoded'] = X['artist_name_split'].apply(lambda artists: encode(artists, artist_popularity))

        return X

In [84]:
ArtistEncoding().fit_transform(data)

,artist_name,year,popularity,album_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,language,artist_name_split,artist_name_encoded
0,Anirudh Ravichander,2024,59,"Leo Das Entry (From ""Leo"")",0.02410,0.753,97297.0,0.970,0.055300,8.0,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,Tamil,[Anirudh Ravichander],22.035066
1,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,AAO KILLELLE,0.08510,0.780,207369.0,0.793,0.000000,10.0,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,Tamil,"[Anirudh Ravichander, Pravin Mani, Vaishali ...",26.289466
2,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,Mayakiriye Sirikiriye (Orchestral EDM),0.03110,0.457,82551.0,0.491,0.000000,2.0,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,Tamil,"[Anirudh Ravichander, Anivee, Alvin Bruno]",19.149466
3,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,Scene Ah Scene Ah (Experimental EDM Mix),0.22700,0.718,115831.0,0.630,0.000727,7.0,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,Tamil,"[Anirudh Ravichander, Bharath Sankar, Kabila...",15.742419
4,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,Gundellonaa X I Am a Disco Dancer (Mashup),0.01530,0.689,129621.0,0.748,0.000001,7.0,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,Tamil,"[Anirudh Ravichander, Benny Dayal, Leon Jame...",17.775819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62312,"Arvind Raj, Sheezay, Music Kitchen, FSPROD Vin...",2021,30,Sani (G.O.A.T Remix),0.08490,0.877,359718.0,0.469,0.000000,7.0,0.1100,-11.458,1.0,0.3530,100.024,4.0,0.504,Tamil,"[Arvind Raj, Sheezay, Music Kitchen, FSPROD...",29.662500
62313,"A H Kaashif, Navakkarai Naveen Prabanjam, Asal...",2021,30,Bachelor (Original Motion Picture Soundtrack),0.07880,0.833,154364.0,0.689,0.000000,8.0,0.2670,-5.061,1.0,0.0813,110.041,4.0,0.524,Tamil,"[A H Kaashif, Navakkarai Naveen Prabanjam, A...",29.375000
62314,"Rakesh Ambigapathy, Asal Kolaar, MC Vickey",2021,18,Yo Baby,0.00918,0.842,234000.0,0.765,0.000000,7.0,0.0790,-5.855,1.0,0.1370,122.038,4.0,0.669,Tamil,"[Rakesh Ambigapathy, Asal Kolaar, MC Vickey]",16.788978
62315,"Asal Kolaar, Priyadarshan Balasubramanian",2021,1,Arjuna Phalguna,0.10100,0.682,180807.0,0.861,0.000000,1.0,0.3200,-6.061,1.0,0.2850,74.993,4.0,0.439,Tamil,"[Asal Kolaar, Priyadarshan Balasubramanian]",11.653846


Frequency encoding

In [120]:
class FrequencyEncoder(ETL):

    TARGET_COL = 'artist_name'

    def __init__(self):
        self.frequencies = None
        self.default_value = 0

    
    def fit(self, X, y=None):
        freqs = X[self.TARGET_COL].str.split(',').explode().value_counts()
        self.frequencies = freqs.to_dict()
        return self
    

    def transform(self, X, y=None):

        X_new = X.copy()

        X_new[self.TARGET_COL] = X[self.TARGET_COL].map(self.frequencies).fillna(self.default_value)

        return X_new


Key column transformation - cyclical data capturing:

Original mapping is linear - relationship of the keys is not correctly represented, according to the data model description  
the mapping represents the keys follows:

$$
\begin{array}{|c|c|}
\hline
\textbf{Key} & \textbf{Pitch Class} \\ \hline
0 & C \\ \hline
1 & C\# / D\flat \\ \hline
2 & D \\ \hline
3 & D\# / E\flat \\ \hline
4 & E \\ \hline
5 & F \\ \hline
6 & F\# / G\flat \\ \hline
7 & G \\ \hline
8 & G\# / A\flat \\ \hline
9 & A \\ \hline
10 & A\# / B\flat \\ \hline
11 & B \\ \hline
\end{array}
$$

We will first transform the original column to the correct representation on the circle of fifths, then remap the correct ordinal representation  
to the circle of fifths trigonometric representation defined as:

$\theta = \frac{2\pi}{12}.key$

Then we can create correct encoding representation with two columns defined as:

$x = \cos(\theta)$, $y = \sin(\theta)$

In [15]:
class CircleOfFifthsEncoding(ETL):

    TARGET_COLUMN = 'key'

    # Pôvodný mapping z datasetu (chromatic scale: 0–11)
    # Tento mapping je založený na poltónových intervaloch.
    chromatic_scale = {
        0: "C",
        1: "C#/Db",
        2: "D",
        3: "D#/Eb",
        4: "E",
        5: "F",
        6: "F#/Gb",
        7: "G",
        8: "G#/Ab",
        9: "A",
        10: "A#/Bb",
        11: "B"
    }

    # Korektný ordinal pre Circle of Fifths:
    # Táto sekvencia reflektuje harmonické vzťahy medzi kľúčmi:
    # každý nasledujúci kľúč je vzdialený o kvintu (7 poltónov).
    circle_of_fifths_mapping = {
        0: 0,  # C
        7: 1,  # G
        2: 2,  # D
        9: 3,  # A
        4: 4,  # E
        11: 5, # B
        6: 6,  # F#/Gb
        1: 7,  # C#/Db
        8: 8,  # G#/Ab
        3: 9,  # D#/Eb
        10: 10, # A#/Bb
        5: 11  # F
    }


    def transform(self, X, y=None):

        X_new = X.copy()

        ordinal_remapping = X[self.TARGET_COLUMN].map(self.circle_of_fifths_mapping)

        theta = (2 * np.pi / 12) * ordinal_remapping

        x = np.cos(theta)
        y = np.sin(theta)

        X_new = X_new.drop(self.TARGET_COLUMN, axis=1, errors='ignore')

        X_new['key_x'] = x
        X_new['key_y'] = y

        return X_new

In [16]:
CircleOfFifthsEncoding().fit_transform(data).head()

,artist_name,year,popularity,album_name,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,mode,speechiness,tempo,time_signature,valence,language,artist_name_split,key_x,key_y
0,Anirudh Ravichander,2024,59,"Leo Das Entry (From ""Leo"")",0.0241,0.753,97297.0,0.970,0.055300,0.1000,-5.994,0.0,0.1030,110.997,4.0,0.459,Tamil,[Anirudh Ravichander],-0.500000,-0.866025
1,"Anirudh Ravichander, Pravin Mani, Vaishali Sri...",2024,47,AAO KILLELLE,0.0851,0.780,207369.0,0.793,0.000000,0.0951,-5.674,0.0,0.0952,164.995,3.0,0.821,Tamil,"[Anirudh Ravichander, Pravin Mani, Vaishali ...",0.500000,-0.866025
2,"Anirudh Ravichander, Anivee, Alvin Bruno",2024,35,Mayakiriye Sirikiriye (Orchestral EDM),0.0311,0.457,82551.0,0.491,0.000000,0.0831,-8.937,0.0,0.1530,169.996,4.0,0.598,Tamil,"[Anirudh Ravichander, Anivee, Alvin Bruno]",0.500000,0.866025
3,"Anirudh Ravichander, Bharath Sankar, Kabilan, ...",2024,24,Scene Ah Scene Ah (Experimental EDM Mix),0.2270,0.718,115831.0,0.630,0.000727,0.1240,-11.104,1.0,0.4450,169.996,4.0,0.362,Tamil,"[Anirudh Ravichander, Bharath Sankar, Kabila...",0.866025,0.500000
4,"Anirudh Ravichander, Benny Dayal, Leon James, ...",2024,22,Gundellonaa X I Am a Disco Dancer (Mashup),0.0153,0.689,129621.0,0.748,0.000001,0.3450,-9.637,1.0,0.1580,128.961,4.0,0.593,Tamil,"[Anirudh Ravichander, Benny Dayal, Leon Jame...",0.866025,0.500000


### Standardization - Scaling

In [17]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

We will use native sklearn scalers such as:
- standard scaler
- min max scaler

### Filtration of columns

- handling of utility columns during transformation sub-steps & EDA drop columns

In [18]:
class FilterColumns(ETL):
    
    def __init__(self, drop_columns=None):

        if drop_columns is None:
            drop_columns = []

        self.drop_columns = drop_columns


    def transform(self, X, y=None):
        X_new = X.copy()
        X_new = X.drop(self.drop_columns, axis=1, errors='ignore')
        return X_new

### NaN Handling

We will utilize KNN Imputer/Dropping Nulls

- NaNs are represented by -1

In [19]:
data.min()

artist_name             "Karaoke - Britney Spears, Madonna"
year                                                   1971
popularity                                                0
album_name                          "...von Bach bis AC/DC"
acousticness                                           -1.0
danceability                                           -1.0
duration_ms                                          5000.0
energy                                                 -1.0
instrumentalness                                       -1.0
key                                                    -1.0
liveness                                               -1.0
loudness                                          -100000.0
mode                                                   -1.0
speechiness                                            -1.0
tempo                                                  -1.0
time_signature                                         -1.0
valence                                 

In [20]:
nan_columns = ['acousticness', 'danceability', 'energy', 'liveness', 'speechiness', 'tempo', 'valence']

In [21]:
null_replaced = data[nan_columns].replace(-1, np.nan)

In [22]:
class ConvertNull(ETL):
    
    def __init__(self, input_value=-1, output_value=np.nan):
        self.input_value = input_value
        self.output_value = output_value

    
    def transform(self, X):
        X_new = X.copy()
        X_new = X_new.replace(self.input_value, self.output_value)
        return X_new

### ETL Pipeline Construction

NOTE: akurat pozeram ze Artist popularity aggregation nieje mozne vyuzit... popularita je target parameter ktori budeme predikovat cize pri test sete by nebolo ako vypocitat  
tie priemery... nechapem ako mi to nedoslo skorej.

In [23]:
import scipy.stats

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import PowerTransformer

from sklearn.preprocessing import PolynomialFeatures

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV

pipeline construction

In [24]:
data.columns

Index(['artist_name', 'year', 'popularity', 'album_name', 'acousticness',
       'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key',
       'liveness', 'loudness', 'mode', 'speechiness', 'tempo',
       'time_signature', 'valence', 'language', 'artist_name_split'],
      dtype='object')

In [25]:
numeric_columns = list(data.columns[data.dtypes != object].difference(['key', 'mode', 'popularity']))
categorical_columns = list(data.columns[data.dtypes == object])
acustic_categorical_columns = ['key', 'mode']

nan_columns = ['acousticness', 'danceability', 'energy', 'liveness', 'speechiness', 'tempo', 'valence']
drop_columns = ['artist_name_split', 'track_id', 'artwork_url', 'track_url', 'track_name']

target_column = 'popularity'

column subset preparation

In [113]:
TARGET = 'popularity'

ONEHOT_COLUMNS = ['language']
CIRCLE_OF_FIFHTS_COLUMNS = ['key']
ARTIST_NAME = ['artist_name']

ALL_CATEGORICAL = ONEHOT_COLUMNS + CIRCLE_OF_FIFHTS_COLUMNS

NUMERIC_COLUMNS = numeric_columns
NAN_COLUMNS = nan_columns
DROP_COLUMNS = drop_columns

In [27]:
X = data.drop(target_column, axis=1)
y = data[target_column]

In [28]:
convert_null = ColumnTransformer(transformers=[
    ('conversion', ConvertNull(), NAN_COLUMNS)
], remainder='passthrough')

In [29]:
numeric_pipeline = Pipeline(steps=[
    ('scaling', StandardScaler())
])

In [121]:
transformations = ColumnTransformer(transformers=[
    
    ('onehot_encoding', OneHotEncoder(sparse_output=False), ONEHOT_COLUMNS),
    ('trigonometric_encoding', CircleOfFifthsEncoding(), CIRCLE_OF_FIFHTS_COLUMNS),
    ('artist_encoding', FrequencyEncoder(), ARTIST_NAME),
    ('nummeric_processing', numeric_pipeline, NUMERIC_COLUMNS)

], remainder='passthrough')

In [122]:
pd.DataFrame(transformations.fit_transform(X))

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.5,-0.866025,1597.0,...,-0.547248,0.024951,0.132681,-0.243228,0.284319,-0.136814,0.992642,"Leo Das Entry (From ""Leo"")",0.0,[Anirudh Ravichander]
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,-0.866025,0.0,...,-0.575731,0.025086,0.064942,1.650825,-1.705114,1.230334,0.992642,AAO KILLELLE,0.0,"[Anirudh Ravichander, Pravin Mani, Vaishali ..."
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.5,0.866025,0.0,...,-0.645487,0.023709,0.566901,1.826242,0.284319,0.388141,0.992642,Mayakiriye Sirikiriye (Orchestral EDM),0.0,"[Anirudh Ravichander, Anivee, Alvin Bruno]"
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.866025,0.5,0.0,...,-0.407736,0.022794,3.102749,1.826242,0.284319,-0.503149,0.992642,Scene Ah Scene Ah (Experimental EDM Mix),1.0,"[Anirudh Ravichander, Bharath Sankar, Kabila..."
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.866025,0.5,0.0,...,0.87693,0.023413,0.610323,0.386884,0.284319,0.369258,0.992642,Gundellonaa X I Am a Disco Dancer (Mashup),1.0,"[Anirudh Ravichander, Benny Dayal, Leon Jame..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62312,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.866025,0.5,0.0,...,-0.489118,0.022644,2.303783,-0.628121,0.284319,0.033136,0.681601,Sani (G.O.A.T Remix),1.0,"[Arvind Raj, Sheezay, Music Kitchen, FSPROD..."
62313,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.5,-0.866025,0.0,...,0.423519,0.025345,-0.055771,-0.276761,0.284319,0.108669,0.681601,Bachelor (Original Motion Picture Soundtrack),1.0,"[A H Kaashif, Navakkarai Naveen Prabanjam, A..."
62314,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.866025,0.5,0.0,...,-0.66932,0.02501,0.427951,0.14405,0.284319,0.656283,0.681601,Yo Baby,1.0,"[Rakesh Ambigapathy, Asal Kolaar, MC Vickey]"
62315,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.866025,-0.5,0.0,...,0.731606,0.024923,1.713243,-1.506117,0.284319,-0.212347,0.681601,Arjuna Phalguna,1.0,"[Asal Kolaar, Priyadarshan Balasubramanian]"


In [58]:
data['artist_name'].str.split(',').explode().value_counts().to_dict()

{'Ilaiyaraaja': 3782,
 'A.R. Rahman': 2757,
 'Shankar Mahadevan': 2214,
 'Shreya Ghoshal': 1964,
 'Yuvan Shankar Raja': 1790,
 'Anirudh Ravichander': 1597,
 'Madonna': 1425,
 'Arijit Singh': 1360,
 'Justin Bieber': 1333,
 'Ramin Djawadi': 1333,
 'Alan Silvestri': 1319,
 'Blake Neely': 1274,
 'D. Imman': 1244,
 'Daniel Pemberton': 1198,
 'Pritam': 1178,
 'Maroon 5': 1097,
 'Deva': 1086,
 ' S. P. Balasubrahmanyam': 1021,
 'Taylor Swift': 1011,
 'Billy Joel': 969,
 ' K. S. Chithra': 968,
 'The Weeknd': 915,
 'G. V. Prakash': 912,
 'Vijay Yesudas': 837,
 'Javed Ali': 814,
 'Bruno Mars': 812,
 'Harris Jayaraj': 795,
 'Hans Zimmer': 783,
 ' S. Janaki': 738,
 'Thaman S': 729,
 ' Mano': 680,
 ' Shreya Ghoshal': 656,
 'Santhosh Narayanan': 641,
 ' Shakira Shute': 585,
 'Keri Beevis': 585,
 'Vijay Prakash': 581,
 'TWICE': 578,
 'Shankar-Ehsaan-Loy': 565,
 'Rihanna': 565,
 '*NSYNC': 559,
 'SUPER JUNIOR': 536,
 ' Arijit Singh': 533,
 'Linkin Park': 507,
 'Coldplay': 501,
 'Shakira': 494,
 'SHINee'

model selection

In [9]:
model = None